In [6]:
import numpy as np 
import networkx as nx
from itertools import permutations, combinations

## Problem 1: Suggesting Similar Papers
A citation network is a directed network where the vertices are academic papers and there is a directed edge from paper 𝐴 to paper 𝐵 if paper 𝐴 cites paper 𝐵 in its bibliography. Google Scholar performs automated citation indexing and has a useful feature that allows users to find similar papers. In the following, we analyze two approaches for measuring similarity between papers.

### Part (a): Co-citation network
- Two papers are said to be cocited if they are both cited by the same third paper. 
- The edge weights in the cocitation network correspond to the number of cocitations. 
In this part, we will discover how to compute the (weighted) adjacency matrix of the cocitation network from the adjacency matrix of the citation network.
- Problem setup: In order to derive the cocitation matrix, we need to derive it as a function of the original adjacency matrix.
- Problem notation: If there is an edge from paper 𝑖 to paper 𝑗, it means that paper 𝑖 cites paper 𝑗. We will denote by 𝐴 the corresponding adjacency matrix, such that 𝐴𝑖𝑗=1 means there is a directed edge from 𝑖 to 𝑗. Let us denote by 𝐶 the cocitation network matrix.

#### Question 1
In attempting to derive the cocitation matrix, your friend came up with the following algorithm:
- Assuming the row indices of the matrix mean that the paper is citing others, and the column indices that the paper is being cited, then the algorithm's steps would be:
    1. Construct an empty matrix for 𝐶.
    2. Go through the rows of 𝐴 one by one.
    3. For each row 𝑟 of 𝐴, if the row sum is strictly greater than 1, then do: for each pair ((𝑟,𝑎),(𝑟,𝑏)) in row 𝑟 that are non-zero (meaning that there is an existing relationship), add 1 to 𝐶 at the location (𝑎,𝑏). Note that by following this rule, you will naturally also add 1 to 𝐶 at location (𝑏,𝑎) as the pair ((𝑟,𝑏),(𝑟,𝑎)) must also be present.

After reading carefully through the proposed steps, please answer the following:

Does this generate the cocitation weighted adjacency matrix?
- Ans
    - Yes

What is the big-O complexity, $\mathcal{O}$, of the proposed algorithm, in terms of 𝑛, the number of nodes in the graph?
- Ans 
    - $\mathcal{O}(n^3)$
    
``` Given Adjacency matrix = A
init cocitation network matrix C = np.empty(shape=adj.shape)

For i_row in A: --> n 
    if sum(i_row, axis=0) > 1:
        extra_weight_links = combinations(where(i_row>0),2) --> n(n-1)/2 --> n^2
        for a,b in link fo link for in extra_weight_links:
            C[a,b] +=1
            C[b,a] +=1
```

#### Question 2

Write the cocitation weighted adjacency matrix, 𝐶, in terms of 𝐴 using matrix operations. Use A^T for 𝐴⊺ and * for matrix multiplication. The diagonals in your answer need not match the diagonals generated by the definition in Question 1, the off-diagonals should match Question 1.
- Ans
    - By the definition of notation
        - ''If there is an edge from paper 𝑖 to paper 𝑗, it means that paper 𝑖 cites paper 𝑗. We will denote by 𝐴 the corresponding adjacency matrix, such that 𝐴𝑖𝑗=1''
        - $A_{i,j} \in \{0,1\}$
        1. update rule in Q1
            - Condition 1, if $\sum\limits_{j=1}^{N}(A_{i,j})>0$, do
                - Condition 2, if $A_{i,a}$ > 0 and $A_{i,b}$ > 0  | $a\neq b$ and $a,b \in \{1,\ldots,N\}$, do
                    - $C_{a,b} +=1$
        2. re-think Condition 2
            - $A_{i,a}$ and $A_{i,b}$>0   -->   0<$A_{i,a} \cdot A_{i,b}$=1
            - if $A_{i,a} \cdot A_{i,b}$ = 1, do
                - $C_{a,b} += A_{i,a} \cdot A_{i,b}$
            - $C_{a,b} = \sum\limits_{i=1}^{N}(A_{i,a} \cdot A_{i,b})$
            - $C_{a,b} = col_{a}^T \cdot col_{b}$ | $a\neq b$ and $col_{a}, col_{b} \in A$
            - $\begin{align}
                C &= A^TA - (A^TA \circ I) & (X \circ Y)\text{: Hadamard product, element wise multiplication }\\
                  &= A^TA - diag( ({A^TA})_{i,i} \text{ for }i \in \{1,\ldots,N\})
                  \end{align}$
            


### Part (b): Bibliographic coupling

Two papers are said to be bibliographically coupled if they cite the same other papers. The edge weights in a bibliographic coupling correspond to the number of common citations between two papers.

How do you compute the (weighted) adjacency matrix of the bibliographic coupling, 𝐵, from the adjacency matrix of the citation network, 𝐴? Write your answer in terms of matrix operations.
- Ans
    - $B_{a,b} = row_{a}^T \cdot row_{b}$ | $a\neq b$ and $row_{a}, row_{b} \in A$
    - $B = AA^T - (AA^T \circ I)$

### Part (c):   (2 points)  Include your answer to this question in your written report. (100 word limit.)

How does the time complexity of your solution involving matrix multiplication in part (a) compare to your friend's algorithm?
- refs
    - key words 
        - In practice, everybody uses Lapack's DGEMM.
        - DGEMM is in fact much faster (it uses blocking for instance that helps caching) than the three nested loops people learn in basic algorithm classes, but the complexity is still the same:
            - The person reference for BLAS and LAPACK is Prof. Jack Dongarra who has a nice set of slides here:
                - [Linear Algebra Algorithms](http://www.netlib.org/utk/people/JackDongarra/WEB-PAGES/SPRING-2005/Lect07.pdf)
        - Incidentally, DGEMM and BLAS in general are for dense linear algebra. For sparse matrices (like in these cases here), the complexity will better if you consider the non-zero entries. But again, in the worst-case (in which we have a dense matrix) the complexity is pretty much the same.

### Part (d):   (3 points)  Include your answer to this question in your written report. (200 word limit.)

Bibliographic coupling and cocitation can both be taken as an indicator that papers deal with related material. However, they can in practice give noticeably different results. Why? Which measure is more appropriate as an indicator for similarity between papers?

- C_a,b : paper a and paper b are pats of paper i
    - paper i = {paper a, paper b, ...}
- B_a,b : paper a and paper b both has paper i as it part
    - paper a = {paper i, ...}, paper b = {paper i, ...}

## Problem 2: Investigating a time-varying criminal network
- [Information and data description](https://learning.edx.org/course/course-v1:MITx+6.419x+1T2021/block-v1:MITx+6.419x+1T2021+type@sequential+block@networks_report/block-v1:MITx+6.419x+1T2021+type@vertical+block@networks_report-tab3)
- [paper](https://www.researchgate.net/publication/292304919_Modeling_Verdict_Outcomes_Using_Social_Network_Measures_The_Watergate_and_Caviar_Network_Cases)
- According to the police, the role of 23 of the players in the “Serero organization" are the following, listed by name (unique id):
    - Daniel Serero (n1) : Mastermind of the network.

    - Pierre Perlini (n3) : Principal lieutenant of Serero, he executes Serero's instructions.

    - Alain (n83) and Gérard (n86) Levy : Investors and transporters of money.

    - Wallace Lee (n85) : Takes care of financial affairs (accountant).

    - Gaspard Lino (n6): Broker in Spain.

    - Samir Rabbat (n11): Provider in Morocco.

    - Lee Gilbert (n88): Trusted man of Wallace Lee (became an informer after the arrest).

    - Beverly Ashton (n106): Spouse of Lino, transports money and documents.

    - Antonio Iannacci (n89): Investor.

    - Mohammed Echouafni (n84): Moroccan investor.

    - Richard Gleeson (n5), Bruno de Quinzio (n8) and Gabrielle Casale (n76) : Charged with recuperating the marijuana.

    - Roderik Janouska (n77): Individual with airport contacts.

    - Patrick Lee (n87): Investor.

    - Salvatore Panetta (n82): Transport arrangements manager.

    - Steve Cunha (n96): Transport manager, owner of a legitimate import company (became an informer after the arrest).

    - Ernesto Morales (n12): Principal organizer of the cocaine import, intermediary between the Colombians and the Serero organization.

    - Oscar Nieri (n17): The handyman of Morales.

    - Richard Brebner (n80): Was transporting the cocaine from the US to Montréal.

    - Ricardo Negrinotti (n33): Was taking possession of the cocaine in the US to hand it to Brebner.

    - Johnny Pacheco (n16): Cocaine provider.

In [10]:
import pandas as pd
import networkx as nx
phases = {}
G = {}
for i in range(1,12): 
    var_name = "phase" + str(i)
    file_name = "https://raw.githubusercontent.com/ragini30/Networks-Homework/main/" + var_name + ".csv"
    phases[i] = pd.read_csv(file_name, index_col = ["players"])
    phases[i].columns = "n" + phases[i].columns
    phases[i].index = phases[i].columns
    G[i] = nx.from_pandas_adjacency(phases[i])
    G[i].name = var_name

### Part (a) 
#### Question 1
What is the size of the network at each phase? Plot the evolution of the number of node and number of edges over time, from phase 1 to 11.
Provide the number of nodes and edges for the three phases listed below: [2,6,10]

In [24]:
for phase in [2,6,10]:
    print('Phase', phase)
    print('number of nodes = ', len(G[phase].nodes))
    print('number of edges = ', len(G[phase].edges))

Phase 2
number of nodes =  24
number of edges =  28
Phase 6
number of nodes =  27
number of edges =  47
Phase 10
number of nodes =  42
number of edges =  50


#### Question 2
Try visualizing the graph at each phase. For networkx you can use

```nx.draw(g, pos=nx.drawing.nx_agraph.graphviz_layout(g), with_labels=True)  ```

where the graphviz layout algorithm graphviz_layout has been used.

The graphviz algorithm is recommended for these complex graphs, and you will need it to answer some of these questions. 

For macOS and Windows, you can find instructions here.
```pip install pygraphviz```


Visualize the graph for Phase 3. Which of the following plots below correspond to Phase 3?
(can't install pygraphviz, move .ipynb to colab)

In [26]:
nx.draw(G[3], pos=nx.drawing.nx_agraph.graphviz_layout(G[3]), with_labels=True)  

ImportError: requires pygraphviz http://pygraphviz.github.io/